In [1]:
import psycopg2
from psycopg2 import sql
import pandas as pd

In [2]:
def create_db(dbname):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
            database='postgres',
            user='postgres',
            password='password',
            host='localhost',
            port='5432'
        )
        
        connection.autocommit = True
        cursor = connection.cursor()
        
        # Create database using psycopg2.sql to safely construct the SQL statement
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(dbname)))
        
        print(f"Database {dbname} created successfully.")

    except Exception as e:
        print(f"Operation failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def drop_db(dbname):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
            database='postgres',
            user='postgres',
            password='password',
            host='localhost',
            port='5432'
        )
        
        connection.autocommit = True
        cursor = connection.cursor()
        
        # Create database using psycopg2.sql to safely construct the SQL statement
        cursor.execute(sql.SQL("DROP DATABASE {}").format(sql.Identifier(dbname)))
        
        print(f"Database {dbname} dropped successfully.")

    except Exception as e:
        print(f"Operation failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def transition(query, params=None, dbname='krannet'):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
        database=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port= '5432')
        
        cursor = connection.cursor()

        # Transition
        cursor.execute(sql.SQL('BEGIN'))

        if not params:
            cursor.execute(sql.SQL(query))
        else: 
            cursor.execute(sql.SQL(query), params)
        
        # Commit transition
        connection.commit()

        print("Transaction committed successfully.")

    except Exception as e:
        connection.rollback()
        print(f"Transaction failed: {e}")

    finally:
        if cursor:
            cursor.close()
        if connection:
            connection.close()

def fetch_table_data(table_name, dbname='krannet'):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
        database=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port= '5432')
        
        # Fetch data using pandas
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(query, connection)
        
        return df

    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None

    finally:
        if connection:
            connection.close()

def fetch_data(query, dbname='krannet'):
    try:
        # Connect to your PostgreSQL database
        connection = psycopg2.connect(
        database=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port= '5432')
        
        # Fetch data using pandas
        df = pd.read_sql_query(query, connection)
        
        return df

    except Exception as e:
        print(f"Failed to fetch data: {e}")
        return None

    finally:
        if connection:
            connection.close()

Some usefull SQL prommt:
- `CREATE database <db_name>`
- `DROP database <db_name>`

In [3]:
# Env Variables
DB_NAME = 'krannet'

# Drop DB
drop_db(DB_NAME)

Database krannet dropped successfully.


## Create DB

In [4]:
# Env Variables
DB_NAME = 'krannet'

# Create DB
create_db(DB_NAME)

Database krannet created successfully.


## Create Tables

In [5]:
# SQL queries to create the tables
create_meeting_table = """
CREATE TABLE IF NOT EXISTS meeting_table (
    meeting_id SERIAL PRIMARY KEY,
    title VARCHAR(100) NOT NULL,
    year INT NOT NULL
);
"""

old_create_speaker_table = """
CREATE TABLE IF NOT EXISTS speaker_table (
    speaker_id SERIAL PRIMARY KEY,
    meeting_id INT NOT NULL,
    speaker_name VARCHAR(50) NOT NULL,
    interventions SMALLINT NOT NULL,
    FOREIGN KEY (meeting_id) REFERENCES meeting_table(meeting_id)
);
"""

create_speaker_table = """
CREATE TABLE IF NOT EXISTS speaker_table (
    speaker_id SERIAL PRIMARY KEY,
    meeting_id INT NOT NULL,
    speaker_name VARCHAR(50) NOT NULL,
    interventions SMALLINT NOT NULL
);
"""

# Execute the transition function to create the tables
transition(create_meeting_table)
transition(create_speaker_table)

Transaction committed successfully.
Transaction committed successfully.


In [6]:
# insert data into meeting_table

meeting_id = 0
titles = ['meeting1', 'meeting2']
years = [1990, 1990]

for t, y in zip(titles, years):
    query = 'INSERT INTO meeting_table (title, year) VALUES (%s, %s)'
    params = (t, y)
    transition(query, params)


    # insert data into speaker_table
    meeting_id += 1
    speakers = ['President', 'Alberto']
    interventions = [30, 22]

    for s, i in zip(speakers, interventions):
        query = 'INSERT INTO speaker_table (meeting_id, speaker_name, interventions) VALUES (%s, %s, %s)'
        params = (meeting_id, s, i)
        transition(query, params)

Transaction committed successfully.
Transaction committed successfully.
Transaction committed successfully.
Transaction committed successfully.
Transaction committed successfully.
Transaction committed successfully.


In [7]:
df = fetch_table_data(table_name='meeting_table')
df.head()

/var/folders/nv/ljsvt6pd7v526ds10294vq_w00011d/T/ipykernel_91449/1521635581.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,meeting_id,title,year
0,1,meeting1,1990
1,2,meeting2,1990


In [8]:
df = fetch_table_data(table_name='speaker_table')
df.head()

/var/folders/nv/ljsvt6pd7v526ds10294vq_w00011d/T/ipykernel_91449/1521635581.py:104: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,speaker_id,meeting_id,speaker_name,interventions
0,1,1,President,30
1,2,1,Alberto,22
2,3,2,President,30
3,4,2,Alberto,22


## Access data

In [9]:
query = """
SELECT DISTINCT
    M.year AS year,
    S.*
FROM 
    meeting_table M
JOIN 
    speaker_table S ON M.meeting_id = S.meeting_id
WHERE 
    M.year = '1990';
"""

df = fetch_data(query)
df.head()


/var/folders/nv/ljsvt6pd7v526ds10294vq_w00011d/T/ipykernel_91449/1521635581.py:127: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


,year,speaker_id,meeting_id,speaker_name,interventions
0,1990,1,1,President,30
1,1990,2,1,Alberto,22
2,1990,3,2,President,30
3,1990,4,2,Alberto,22
